In [ ]:
#STUDENT NAME: SHRUTHI
#STUDENT ID: 801218392
#HOMEWORK 3 QUESTION 1-B

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from datetime import datetime

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [2]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [3]:
from torchvision import datasets, transforms
data_path = 'C:/Users/Shruthi/Documents/HWs/RTML/hw2/data'
# Here the training set is downloaded as cifar10

cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [4]:
# The validation set is downloaded as cifar10_val
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [5]:
#The training set is defined as train_loader
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)

In [6]:
#The validation set is defined as val_loader
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

In [7]:
model2 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(16, 8, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),
            nn.Linear(512, 64),
            nn.LogSoftmax(dim=1))

In [8]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):

    correctt = 0
    totalt = 0
    
    for epoch in range (1, n_epochs+1):
        loss_train=0.0
        for imgs, labels in train_loader:
            outputs= model(imgs)
            loss=loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train+=loss.item()
            
        if epoch ==1 or epoch % 10==0:
            now = datetime.now()
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
            print(dt_string, "Epoch: %d, Loss: %f" % (epoch, float(loss)))
            
        with torch.no_grad():
            for imgs, labels in train_loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
            totalt += labels.shape[0]
            correctt += int((predicted == labels).sum())
            acctt = float(correctt/totalt)
    print("Accuracy in Training: ",acctt)

In [9]:
optimizer = optim.SGD(model2.parameters(), lr=1e-2)
loss_fn = nn.NLLLoss()

training_loop(
    n_epochs = 300,
    optimizer = optimizer,
    model = model2,
    loss_fn = loss_fn,
    train_loader = train_loader)

25/03/2022 08:15:21 Epoch: 1, Loss: 2.309780
25/03/2022 08:20:04 Epoch: 10, Loss: 1.673684
25/03/2022 08:25:20 Epoch: 20, Loss: 1.557179
25/03/2022 08:30:43 Epoch: 30, Loss: 1.400070
25/03/2022 08:35:58 Epoch: 40, Loss: 1.355903
25/03/2022 08:41:21 Epoch: 50, Loss: 1.309954
25/03/2022 08:46:37 Epoch: 60, Loss: 1.245668
25/03/2022 08:52:07 Epoch: 70, Loss: 1.203056
25/03/2022 08:57:36 Epoch: 80, Loss: 1.177593
25/03/2022 09:03:02 Epoch: 90, Loss: 1.153771
25/03/2022 09:08:38 Epoch: 100, Loss: 1.123013
25/03/2022 09:14:04 Epoch: 110, Loss: 1.114730
25/03/2022 09:19:31 Epoch: 120, Loss: 1.108476
25/03/2022 09:24:52 Epoch: 130, Loss: 1.084753
25/03/2022 09:30:04 Epoch: 140, Loss: 1.080791
25/03/2022 09:35:18 Epoch: 150, Loss: 1.064143
25/03/2022 09:40:21 Epoch: 160, Loss: 1.050926
25/03/2022 09:45:41 Epoch: 170, Loss: 1.045110
25/03/2022 09:51:13 Epoch: 180, Loss: 1.044679
25/03/2022 09:56:43 Epoch: 190, Loss: 1.038527
25/03/2022 10:02:05 Epoch: 200, Loss: 1.031460
25/03/2022 10:07:32 Epoc

In [10]:
def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correctv = 0
        totalv = 0
        
        with torch.no_grad():
            for imgs, labels in val_loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
            totalv += labels.shape[0]
            correctv += int((predicted == labels).sum())
            acctv = float(correctv/totalv)
    print("Accuracy in Validation: ",acctv)
validate (model2, train_loader, val_loader)

Accuracy in Validation:  0.625


In [11]:
import torchvision.models as models
from ptflops import get_model_complexity_info

net = model2
macs, params = get_model_complexity_info(net, (3, 32, 32), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Sequential(
  0.034 M, 100.000% Params, 0.001 GMac, 100.000% MACs, 
  (0): Conv2d(0.0 M, 1.301% Params, 0.0 GMac, 55.579% MACs, 3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(0.0 M, 0.000% Params, 0.0 GMac, 1.985% MACs, )
  (2): MaxPool2d(0.0 M, 0.000% Params, 0.0 GMac, 1.985% MACs, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(0.001 M, 3.368% Params, 0.0 GMac, 35.977% MACs, 16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU(0.0 M, 0.000% Params, 0.0 GMac, 0.248% MACs, )
  (5): MaxPool2d(0.0 M, 0.000% Params, 0.0 GMac, 0.248% MACs, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, start_dim=1, end_dim=-1)
  (7): Linear(0.033 M, 95.331% Params, 0.0 GMac, 3.978% MACs, in_features=512, out_features=64, bias=True)
  (8): LogSoftmax(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, dim=1)
)
Computational complexity:       0.0 GMac
Number of paramet

In [12]:
numel_list = [p.numel() for p in model2.parameters()]
sum(numel_list), numel_list

(34440, [432, 16, 1152, 8, 32768, 64])